In [ ]:
import sys
sys.path.append("../")

import nltk
nltk.download("punkt") #honestly idek what this is but seems like I have to do it
import os
import matplotlib.pyplot as plt
import numpy as np
import datasets
import torch
import evaluate
import time
import urllib.request as libreq
from training_helpers import *
from generate import Generator
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModel, \
Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq, PegasusModel


In [ ]:
#BUILDING DATASETS

def make_arxiv_dataset():
    start = time.time()
    sample_indices = np.random.randint(low = 0, high = len(metadata), size = 10)
    sampled_articles = ["" for _ in range(sample_size)]
    sampled_ids = ["" for _ in range(sample_size)]
    for ii, i in enumerate(sample_indices):
        article_id = metadata["id"][i]
        sampled_ids[ii] = article_id
        b = req_article(article_id)
        sampled_articles[ii] = b.decode()
    end = time.time()

    save_article_list(sampled_articles, "all_articles.txt")
    end = time.time()
    print(end-start)
    
def make_wiki_dataset():
    #wiki sentences into lists
    with open(NORMAL_WIKI_PATH, "rt") as f:
        normal = f.readlines()
    with open(SIMPLE_WIKI_PATH, "rt") as f:
        simple = f.readlines()
    #collect titles and articles
    normal_ds = {}
    curr_sentences = []
    curr_title = normal[0].split("\t")[0] #initialize with first title
    for a in normal:
        split = a.split("\t")
        title, para, sentence = split[0], split[1], split[2][:-1] #drop new line character

        if title != curr_title:
            art = " ".join(curr_sentences)
            normal_ds[curr_title] = art
            curr_title = title
            curr_sentences = []        
        curr_sentences.append(sentence)

    art = " ".join(curr_sentences)
    normal_ds[curr_title] = art



    #simple wiki
    simple_ds = {}
    curr_sentences = []
    curr_title = simple[0].split("\t")[0] #initialize with first title
    for a in simple:
        split = a.split("\t")
        title, para, sentence = split[0], split[1], split[2][:-1] #drop new line character

        if title != curr_title:
            art = " ".join(curr_sentences)
            simple_ds[curr_title] = art
            curr_title = title
            curr_sentences = []
        curr_sentences.append(sentence)

    art = " ".join(curr_sentences)
    simple_ds[curr_title] = art

    titles = []
    s_articles = []
    n_articles = []
    i = 0
    for k in normal_ds.keys():
        try:
            s_articles.append(simple_ds[k])
            n_articles.append(normal_ds[k])
            titles.append(k)
        except: 
            print("error thrown, probably a non-common key")
    ds = {"titles": titles, "simple_articles": s_articles, "normal_articles": n_articles}
    hf_ds = Dataset.from_dict(ds)
    hf_ds.save_to_disk("../res/datasets/wiki_aligned/aligned_wiki_ds")
    


In [ ]:
#HELPERS

def freeze_base_model_weights(model):
    sd = model.state_dict()
    grad_layers = ["lm_head.weight"]
    c=0
    for k in sd.keys():
        p = sd[k]
        if k in grad_layers:
            p.requires_grad = True
        else: 
            c+=1
            
            p.requires_grad = False
def process_arxiv_src(path, out_dir, article_ids):
    with open(path, "wt") as f:
        pass
    
def save_article_list(sampled_articles,out_path):
    with open(out_path, "wt") as f:
        f.writelines(sampled_articles)

        
def req_article(article_id):
    link = 'http://zzz.cs.cornell.edu:8080/txt_tex?' + article_id
    with libreq.urlopen(link) as url:
      r = url.read()
    return r


def text_to_inputs(ds, tokenizer, text_column, label_column):
    def preprocess_text(batch):
        text_encoding=tokenizer(''.join(batch[text_column]), max_length=1024)
        batch_inputs = text_encoding["input_ids"]
        batch_attentions = text_encoding["attention_mask"]
        
        labels_encoding=tokenizer(''.join(batch[label_column]), max_length=512)
        batch_labels = labels_encoding["input_ids"]
        return {"input_ids": batch_inputs, "attention_mask": batch_attentions, "labels": batch_labels}
    
    return ds.map(preprocess_text, num_proc=4)



def get_text_and_id(arxiv_dir, sample_size=0):
    file_names = []
    file_paths = []
    for root, dirs, files in os.walk(arxiv_dir, topdown=False):
        print(root, dirs, files)
        if root == arxiv_dir:
            for name in files:
                file_names.append(name[:-4])
                file_paths.append(os.path.join(arxiv_dir,name))

    if sample_size:
        sample_indices = np.random.randint(low = 0, high = len(file_names), size = sample_size)   
    else:
        sample_indices = range(len(file_names))
    txt_dataset = {"id": [], "text": []}
    for i in sample_indices:
        name = file_names[i]
        path = file_paths[i]
        with open(path, "r") as f:
            article = f.read()
            txt_dataset["id"].append(name)
            txt_dataset["text"].append(article)
            
    return datasets.Dataset.from_dict(txt_dataset)


def save_list(strings, out_path):
    strings = [strings] if isinstance(strings, str) else strings
    with open(out_path, "wt") as f:
        for i in strings:
            f.write(i + "\n\n\n")
            

class ArxivWikiTrainer(Seq2SeqTrainer):

    def compute_metrics(p):
        m = dataset.load_metric("bertscore")
        preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
        preds = np.argmax(preds, axis=1)
        print("here")
        return {"accuracy": metric_acc.compute(predictions=preds, references=p.label_ids)["accuracy"]}

    """
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")
        # compute custom loss (suppose one has 3 labels with different weights)
        loss_fct = nn.CrossEntropyLoss(weight=torch.tensor([1.0, 2.0, 3.0], device=model.device))
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        print(loss)
        return (loss, outputs) if return_outputs else loss
    """
                                                 


In [ ]:
WORKING_DIR = os.getcwd()
PATH_TO_MODELS = "../res/models/"
PATH_TO_TOKENIZERS = "../res/tokenizers"
MODEL_NAMES = ["bart-base", "pegasus-xsum", "pegasus-x-base-arxiv", "pegasus-x-large", "longformer-base-4096", "led-large-16384-arxiv", "pegasus-x-base"]
MODEL_PATHS = {m: os.path.join(PATH_TO_MODELS, m) for m in MODEL_NAMES}
TOKENIZER_MAPS = {"bart-base": "bart-base", "pegasus-xsum": "pegasus",  "pegasus-x-base-arxiv": "pegasus", "pegasus-x-base": "pegasus", "pegasus-x-large": "pegasus", "longformer-base-4096": "longformer-base-4096", "led-large-16384-arxiv": "longformer-base-4096"}
TOKENIZER_PATHS = {m: os.path.join(PATH_TO_TOKENIZERS, TOKENIZER_MAPS[m]) for m in MODEL_NAMES}
ARXIV_DATA_PATH = "../res/datasets/arxiv_data/arxiv_data_txt/1001"
ARXIV_METADATA_PATH = "../res/datasets/arxiv_data/arxiv-metadata-oai-snapshot.json"
WIKI_PATH = "../res/datasets/wiki_aligned/aligned_wiki_ds"


In [ ]:

    # process_arxiv_src("all_articles.txt", ARXIV_DATA_PATH, article_ids)

In [ ]:
#preparing datasets
metadata = datasets.load_dataset("json", data_files=ARXIV_METADATA_PATH, split="train") #load abstracts
text_and_id = get_text_and_id(ARXIV_DATA_PATH) #raw text ds w/ article ids

labels = ["" for _ in range(len(text_and_id))]
abstracts = metadata.select_columns(["abstract", "id"])
abstracts = {i: abstract for i, abstract in zip(abstracts["id"], abstracts["abstract"])}
for idx, article_id in enumerate(text_and_id["id"]):
    labels[idx] = abstracts[article_id]

text_and_id = text_and_id.add_column("labels", labels)

In [ ]:
text_and_id

In [ ]:
# run training on all models
# TODO: clean up all of the stuff with the splits
MODEL_NAMES = [""]

for name in MODEL_NAMES:
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_PATHS[name], local_files_only=True)
    model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_PATHS[name], local_files_only=True).to(device)
    # pipe = transformers.pipeline("summarization", model=model, tokenizer=tokenizer)

    
    #data
    ds = text_to_inputs(text_and_id, tokenizer, "text", "labels").remove_columns(["id", "text"])
    print(ds.features)
    splits = ds.train_test_split(test_size=0.1)
    train_ds = splits["train"]
    eval_ds = splits["test"]
    data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="pt")
    
    #prep model
    freeze_base_model_weights(model)
    model.enable_input_require_grads()
    model.train()
    
    #trainer
    train_arg_dict = {"output_dir": "../out/" + str(name) + "/", "save_steps": 100, "evaluation_strategy": "steps",\
                      "eval_steps": 100, "logging_steps": 50,  "max_steps": 600, "per_device_train_batch_size": 1}
    train_args = Seq2SeqTrainingArguments(**train_arg_dict)
    trainer = ArxivWikiTrainer(model, args=train_args, train_dataset=train_ds, eval_dataset = eval_ds, tokenizer=tokenizer, data_collator=data_collator)
    trainer.train()
    
    del tokenizer, model, ds, splits, train_ds, eval_ds, data_collator, trainer #something causing data leak, not sure what (although this line doenst even help...)
    torch.cuda.empty_cache()
    !nvidia-smi

In [ ]:
!nvidia-smi

In [ ]:
#train#run training on all models
#TODO: clean up all of the stuff with the splits

device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_PATHS["pegasus-x-base"], local_files_only=True).to("cuda:0")

tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_PATHS["pegasus-x-base"], local_files_only=True)
#data
wiki_ds = datasets.load_from_disk(WIKI_PATH)
ds = text_to_inputs(wiki_ds, tokenizer, "normal_articles", "simple_articles")


In [ ]:
#for wiki, basically the same as arxiv
splits = ds.train_test_split(test_size=0.1)
train_ds = splits["train"]
eval_ds = splits["test"].select([i for i in range(20)])
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="pt")

#prep model
freeze_base_model_weights(model)
model.enable_input_require_grads()
model.train()

#trainer
train_arg_dict = {"output_dir": "../out/wiki-pegasus-x-base"  + "/", "save_steps": 100, "evaluation_strategy": "steps",\
                  "eval_steps": 25, "logging_steps": 50,  "max_steps": 500, "per_device_train_batch_size": 1}
train_args = Seq2SeqTrainingArguments(**train_arg_dict)
trainer = ArxivWikiTrainer(model, args=train_args, train_dataset=train_ds, eval_dataset = eval_ds, tokenizer=tokenizer, data_collator=data_collator)
trainer.train()

torch.cuda.empty_cache()
!nvidia-smi

In [ ]:
#THIS IS BENCHMARKING CODE


def get_losses(model_names):
    losses = []
    timesteps = []
    eval_losses = []
    eval_timesteps = []
    for m in model_names:
        losses_model = []
        timesteps_model = []
        eval_losses_model = []
        eval_timesteps_model = []
        path = os.path.join(PATH_TO_CHECKPOINTS, m)
        newest = 0 #most recent checkpoint has logs from every step
        for d in os.listdir(path):
            if d[0: 10] == "checkpoint":
                newest = max(newest, int(d[11:])) #index goes from 10 to 11 because skipping the "-"
                
        newest_path = os.path.join(path, "checkpoint-" + str(newest) + "/trainer_state.json")
        with open(newest_path, "r") as f:
            state = json.load(f)
            #each log is a dict
            for log in state["log_history"]:
                if "eval_loss" in log.keys():
                    eval_losses_model.append(log["eval_loss"])
                    eval_timesteps_model.append(log["step"])
                else:
                    losses_model.append(log["loss"])
                    timesteps_model.append(log["step"])
        
        
        losses.append(losses_model)
        timesteps.append(timesteps_model)
        eval_losses.append(eval_losses_model)
        eval_timesteps.append(eval_timesteps_model)

    return losses, timesteps, eval_losses, eval_timesteps      
                    


def plot_losses(losses, timesteps, model_names, metric_name):
    colors = iter(plt.cm.viridis(np.linspace(0, 1, len(losses))))
    for l, t, c, n in zip(losses, timesteps, colors, model_names):
        plt.plot(t, l, '.', color=c)
        plt.plot(t, l, alpha=1, color=c, label=n)
    
    plt.legend(loc="upper right")
    plt.xlabel("Batches")
    plt.ylabel(metric_name)
    plt.title(metric_name)
    plt.grid()
    plt.show()
    
def save_list(strings, out_path):
    strings = [strings] if isinstance(strings, str) else strings
    with open(out_path, "wt") as f:
        for i in strings:
            f.write(i + "\n\n\n")
          
        
def save_model_outs(articles, model, tokenizer, out_path):
    device = 0 if torch.cuda.is_available() else -1
    pipe = pipeline("summarization", model=model, tokenizer=tokenizer, device=-1)
    summs = [d["summary_text"] for d in pipe(articles)]
    save_list(summs, out_path)

#TODO: LOAD FROM CHECKPOINT FOR AFTER
#given list of articles, save before and after
def before_and_after(articles, model_name):
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_PATHS[model_name], local_files_only=True)
    before_model = AutoModelForSeq2SeqLM.from_pretrained(UNTRAINED_PATHS[model_name], local_files_only=True)
    freeze_weights(before_model)
    before_path = model_name + "before.txt"
    save_model_outs(articles, before_model, tokenizer, before_path)

    after_model = AutoModelForSeq2SeqLM.from_pretrained(TRAINED_PATHS[model_name], local_files_only=True)
    freeze_weights(after_model)
    after_path = model_name + "_after.txt"            
    save_model_outs(articles, after_model, tokenizer, after_path)
    

In [ ]:
#THIS IS THE CODE FOR THE EMBEDDINGS VISUALIZATION

import os
import json
import torch
import transformers
import scipy as sp
import numpy as np
import enchant
import time
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE


def save_json(d, out_path):
    with open(out_path, "wt") as f:
        json.dump(d, f)


def tokenize_and_embed(words, layer):
    global tokenizer_time
    global inference_time
    global use_pooler
    global device

    start = time.time()
    tokens = tokenizer(words, return_tensors="pt", padding=True).to(device)
    end = time.time()
    tokenizer_time += end - start

    start = time.time()
    # shape is [batch size, sequence length, model dim]
    outs = model(**tokens,
                 output_hidden_states=True)  # all hidden layers and initial embedding layer. outs["hidden_states"][-1] == outs["last_hidden_state"]
    if use_pooler:
        embeddings = outs["pooler_output"]
        out = embeddings.cpu().numpy()  # pooler is always correct dim, don't need to do anything else
    else:
        embeddings = outs["hidden_states"][layer][:, 1:-1]  # choose hidden state, and then drop start and end tokens
        out = (torch.mean(embeddings, 1)).squeeze().cpu().numpy()
    end = time.time()
    inference_time += end - start
    return out


def make_vocab(out_path, max_vocab_size, only_alpha=True):
    d = enchant.Dict("en_US")
    vocab = {}
    n_words = 0
    with open("vocab.txt", "rt", errors="replace") as f:
        for line in f:
            if n_words == max_vocab_size:
                break
            else:
                split = line.split()
                word, count = split[0], split[1]
                if only_alpha:
                    if word.isalpha() and d.check(word):
                        vocab[word] = count
                        n_words += 1
                else:
                    vocab[word] = count
                    n_words += 1
    return vocab


# take vocab dict and return dict with dim reduced embeddings
def make_embeddings(vocab, embedder, layer, batch_size=1):
    global sklearn_time
    n_words = len(vocab)
    embeddings = np.zeros((n_words, embedding_dim))
    words = ["" for _ in range(batch_size)]
    for i, k in enumerate(vocab.keys()):
        words[i % batch_size] = k
        nth_word = i + 1
        if (nth_word) % batch_size == 0:  # for now just assume batch_size divides vocab
            e = embedder(words, layer)  # e is numpy array with embeddings for words
            embeddings[nth_word - batch_size: nth_word] = e

    start = time.time()
    pca_embeddings = PCA(n_components=pca_components).fit_transform(embeddings)
    tsne_embeddings = TSNE(n_components=tsne_components).fit_transform(embeddings)
    end = time.time()
    sklearn_time += end - start

    d = {}
    for i, w in enumerate(vocab.keys()):
        x = tsne_embeddings[i][0]
        y = tsne_embeddings[i][1]
        d[w] = {"idx": int(i % 142), "x": float(x),
                "y": float(y)}  # 142 is because there are 142 colors in the json file, will have to address later
    return d


tokenizer_time = 0
inference_time = 0
sklearn_time = 0

if __name__ == "__main__":
    # model = transformers.BertModel.from_pretrained("bert-base-uncased")
    # model.save_pretrained("bert_model")
    # tokenizer = transformers.BertTokenizer.from_pretrained("bert-base-uncased")
    # tokenizer.save_pretrained("bert_tokenizer")
    # max_vocab_size = 96000
    # vocab_path = "vocab_eng_" + str(int(max_vocab_size)) + ".json"
    # vocab_json = make_vocab(vocab_path, max_vocab_size)
    # save_json(vocab_json, vocab_path)

    batch_size = 32
    max_vocab_size = batch_size * 3000
    pca_components = 50
    tsne_components = 2
    embedding_dim = 768
    use_pooler = False
    model_path = "bert_model"
    tokenizer_path = "bert_tokenizer"
    vocab_path = "vocab_eng_" + str(int(max_vocab_size)) + ".json"
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    layers_of_interest = [0, -1, -2, -10]
    for layer in layers_of_interest:
            suffix = "pooler" if use_pooler else str(layer)
            embedding_path = "embeddings_" + str(int(max_vocab_size)) + "_" + suffix + ".json"
            with torch.no_grad():
                model = transformers.AutoModel.from_pretrained(model_path, local_files_only=True).to(device)
                tokenizer = transformers.AutoTokenizer.from_pretrained(tokenizer_path, local_files_only=True)
                with open(vocab_path, "rt") as f:
                    vocab = json.load(f)
                    start = time.time()
                    embeddings = make_embeddings(vocab, tokenize_and_embed, layer, batch_size=batch_size)
                    end = time.time()
                    save_json(embeddings, embedding_path)
                    print("Total embedding time was " + str(end - start))
                    print("sklearn time was" + str(sklearn_time))
                    print("Inference time was" + str(inference_time))
                    print("tokenization time was" + str(tokenizer_time))
